In [ ]:
model_name=""

In [1]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

os.environ['WANDB_DISABLED'] = "true"

In [2]:
# !gdown 1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo
# !gdown 1mq9CWT9MXsH1Y-ihxVL4uEzufQjpBoer
!gdown 1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql

Downloading...
From: https://drive.google.com/uc?id=1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql
To: /content/processed_data.zip
100% 6.00M/6.00M [00:00<00:00, 100MB/s]


In [3]:
!rm -rf processed_data

!unzip -q processed_data.zip

In [4]:
!pip install -q transformers[torch] accelerate -U

!pip install -q datasets

!pip install -q transformers

!pip install -q iterative-stratification

!pip install -q auto-gptq optimum bitsandbytes

In [5]:
import numpy as np

import pandas as pd

import os

from os import path

import glob

from tqdm.notebook import tqdm

from tqdm import tqdm

import string

import copy

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,IntervalStrategy,BitsAndBytesConfig

from datasets import Dataset,load_dataset, DatasetDict

# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



from sklearn.model_selection import train_test_split,KFold

from sklearn.metrics import classification_report,accuracy_score, f1_score

from sklearn.metrics import precision_recall_fscore_support

from sklearn.utils import shuffle

from sklearn.preprocessing import LabelEncoder

from scipy.special import expit

from peft import prepare_model_for_kbit_training

from peft import LoraConfig, get_peft_model,PeftModel


In [6]:
import torch

import torch.nn as nn

import torch.nn.functional as F

In [365]:


models_list_3=[
        model_name,
          ]



models_name_3=[
        model_name,

]


In [428]:
kind="train"

task="track_a"

# langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]
# labels=['Anger','Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
# langs=["amh","arq","ary","chn","deu","esp","hau","hin","ibo","kin","mar","orm","pcm","ptbr","ptmz","ron","rus","som","sun","swa","swe","tat","tir","ukr","vmw","yor","ind","jav","xho","zul"]
langs=["eng"]
# langs=["afr"]


# processed_path=f"processed_data/{kinds[0]}/{tasks[0]}/{langs[3]}.csv"



# train_data=pd.read_csv(processed_path)


In [429]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)




not found: processed_data/train/track_a/ind.csv
not found: processed_data/train/track_a/jav.csv
not found: processed_data/train/track_a/xho.csv
not found: processed_data/train/track_a/zul.csv


In [433]:
train_data=all_csv


In [435]:
train_data=train_data.fillna(0)

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,amh_train_track_a_00001,1,1,0,0,0,0,amh,እናንተ መቸም አትማሩም ምድረ አውሬ ሁላ
1,amh_train_track_a_00002,1,1,0,0,0,0,amh,ምነው የዶክተር እስራኤል ጥላሁን ሞት በዝምታ አለፋችሁት ባለስልጣን ስለሆ...
2,amh_train_track_a_00003,0,0,0,0,0,0,amh,እንዲሁም ይህ እጅግ እርሶን የሚወዶት እና እጅግም የሚሳሳልዎን የአዲስአበ...
3,amh_train_track_a_00004,1,1,0,0,0,0,amh,etv ዘረኛ ናቹሁ አሽቃባጭ ስንት አባቶቻችን መስዋት የሆኑበትን ባዲራ እ...
4,amh_train_track_a_00005,0,1,0,1,0,0,amh,የዛሬው የጦጣ ንክሻ ይለያል ክክ አይ ፊሊፖስ አንተ ጅማታም እንድህ ቅጥል...
...,...,...,...,...,...,...,...,...,...
61103,yor_train_track_a_02988,0,0,0,0,0,0,yor,oju ti pọn awọn fulani ju ni naijiria yii afi ...
61104,yor_train_track_a_02989,1,0,0,0,0,0,yor,á júwe ilé fún niyi akintola tó bá tẹ̀síwájú l...
61105,yor_train_track_a_02990,0,0,0,0,0,0,yor,nigeria vs madagascar super eagles ń gbáradì f...
61106,yor_train_track_a_02991,0,0,0,0,0,0,yor,tb joshua biography tí kìí bá ṣe rélùwéè tó bà...


In [436]:
float_columns = train_data.select_dtypes(include=['float64']).columns
# Convert those columns to int
train_data[float_columns] = train_data[float_columns].astype('int')

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,amh_train_track_a_00001,1,1,0,0,0,0,amh,እናንተ መቸም አትማሩም ምድረ አውሬ ሁላ
1,amh_train_track_a_00002,1,1,0,0,0,0,amh,ምነው የዶክተር እስራኤል ጥላሁን ሞት በዝምታ አለፋችሁት ባለስልጣን ስለሆ...
2,amh_train_track_a_00003,0,0,0,0,0,0,amh,እንዲሁም ይህ እጅግ እርሶን የሚወዶት እና እጅግም የሚሳሳልዎን የአዲስአበ...
3,amh_train_track_a_00004,1,1,0,0,0,0,amh,etv ዘረኛ ናቹሁ አሽቃባጭ ስንት አባቶቻችን መስዋት የሆኑበትን ባዲራ እ...
4,amh_train_track_a_00005,0,1,0,1,0,0,amh,የዛሬው የጦጣ ንክሻ ይለያል ክክ አይ ፊሊፖስ አንተ ጅማታም እንድህ ቅጥል...
...,...,...,...,...,...,...,...,...,...
61103,yor_train_track_a_02988,0,0,0,0,0,0,yor,oju ti pọn awọn fulani ju ni naijiria yii afi ...
61104,yor_train_track_a_02989,1,0,0,0,0,0,yor,á júwe ilé fún niyi akintola tó bá tẹ̀síwájú l...
61105,yor_train_track_a_02990,0,0,0,0,0,0,yor,nigeria vs madagascar super eagles ń gbáradì f...
61106,yor_train_track_a_02991,0,0,0,0,0,0,yor,tb joshua biography tí kìí bá ṣe rélùwéè tó bà...


In [438]:
train_data = train_data[

train_data['clean_message'].isnull()==False]

train_data.reset_index()

,index,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,0,amh_train_track_a_00001,1,1,0,0,0,0,amh,እናንተ መቸም አትማሩም ምድረ አውሬ ሁላ
1,1,amh_train_track_a_00002,1,1,0,0,0,0,amh,ምነው የዶክተር እስራኤል ጥላሁን ሞት በዝምታ አለፋችሁት ባለስልጣን ስለሆ...
2,2,amh_train_track_a_00003,0,0,0,0,0,0,amh,እንዲሁም ይህ እጅግ እርሶን የሚወዶት እና እጅግም የሚሳሳልዎን የአዲስአበ...
3,3,amh_train_track_a_00004,1,1,0,0,0,0,amh,etv ዘረኛ ናቹሁ አሽቃባጭ ስንት አባቶቻችን መስዋት የሆኑበትን ባዲራ እ...
4,4,amh_train_track_a_00005,0,1,0,1,0,0,amh,የዛሬው የጦጣ ንክሻ ይለያል ክክ አይ ፊሊፖስ አንተ ጅማታም እንድህ ቅጥል...
...,...,...,...,...,...,...,...,...,...,...
61103,61103,yor_train_track_a_02988,0,0,0,0,0,0,yor,oju ti pọn awọn fulani ju ni naijiria yii afi ...
61104,61104,yor_train_track_a_02989,1,0,0,0,0,0,yor,á júwe ilé fún niyi akintola tó bá tẹ̀síwájú l...
61105,61105,yor_train_track_a_02990,0,0,0,0,0,0,yor,nigeria vs madagascar super eagles ń gbáradì f...
61106,61106,yor_train_track_a_02991,0,0,0,0,0,0,yor,tb joshua biography tí kìí bá ṣe rélùwéè tó bà...


In [440]:
# label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise']
label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']


In [441]:
train_data['lang'].value_counts()

,count
lang,
pcm,3728
tir,3681
amh,3549
orm,3442
som,3392
swa,3307
yor,2992
ibo,2880
rus,2679


In [442]:
sum(train_data['lang'].isna())

0

In [443]:
nan_rows = train_data[train_data.isna().any(axis=1)]
nan_rows

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message


In [450]:
train_data['clean_message']=train_data['clean_message'].astype(str)

In [451]:
maxi=train_data['clean_message'].apply(len).max()
maxi

4675

In [452]:
train_data[train_data['clean_message'].apply(len) == maxi]

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
28376,orm_train_track_a_02631,1,0,0,0,0,0,orm,pirezidaaanti moootummmaaa naaannnooo oromiyaa...


In [453]:
train_data[train_data['clean_message'].apply(len) == maxi]["clean_message"].values

array(['pirezidaaanti moootummmaaa naaannnooo oromiyaaa obbbooo shimalis abdiiisaaatiiif abiyyy ahamediiif bj dhimmmi rakkkooo caaasefffama gbbb tiiin wolqabateee aanaaa gooorooo dooolaaa irrra gahaaa jiruuu fi qaaama akkka rakkkooo kun hin hiiikamne taaasisuuun moootummmaaa fi uummmata adddaaan fageeesssuuun gabaaasa sobaaa gara keeesssan ergu akkka furmaaatni sirrriiin kennnamuuuf isin beeeksisuuu taa 1gama gabasaaatiiin bulchaaan gbbb obbbooo abdurazaaaqi huseeen hanga hardhaaattti osooo mariii tokkkollle uummmata waliiin hin gagggeeesin osooo tajaaajillli mannneeen hojiii moootummmaaa hin jiraaatin osooo mannneeen barnooota 81 tokkkollle hin saaaqamin osooo ummmatni mormiii caaasefffama gbbb hanga dhumaaattti hanga lubbbuuun jirrruttti hin fudhannnu gadina gujiii teeesssooon isaaa magaaala adooola tae jalattti bullla jedheee ittti fufaaa jiru bulchaaan gbbb obbbooo abdurazaaaqi huseeen gabaaasa sobaaa kan angooo ofiii itttiiin dheeerefatu kan gara biraaatiiin rakkkooon akkka hin hi

In [454]:
# shuffle data
train_data=train_data.sample(frac=1)
# train_data=train_data.sample(100,random_state=42)


In [455]:
# train valid split
# train,valid=train_test_split(train_data,test_size=0.2,random_state=42)

In [456]:
x_train, y_train = train_data['clean_message'].values.tolist(), train_data[label_columns].values.tolist()

In [457]:
y_train_df = pd.DataFrame(y_train, columns=label_columns)

In [458]:
#update label column name

label_distribution = y_train_df.apply(pd.Series.value_counts).T.fillna(0).astype(int)

label_distribution.columns = ['count_0', 'count_1']
label_distribution['sum'] = label_distribution['count_0'] + label_distribution['count_1']
print(label_distribution)

          count_0  count_1    sum
anger       39977     8909  48886
disgust     40258     8628  48886
fear        44857     4029  48886
joy         39311     9575  48886
sadness     39888     8998  48886
surprise    43484     5402  48886


In [459]:
device="cuda" if torch.cuda.is_available() else 'cpu'

In [460]:
from sklearn.utils.class_weight import compute_class_weight

counts_0 = label_distribution['count_0'].to_numpy()
counts_1 = label_distribution['count_1'].to_numpy()

# Compute class-specific weights for each label (each class)
class_weights = []
for i in range(len(counts_0)):
    weight = compute_class_weight('balanced', classes=np.array([0, 1]), y=[0] * counts_0[i] + [1] * counts_1[i])
    class_weights.append(weight)

# Convert the list of weights to a tensor
class_weights_tensor = torch.tensor([w[1] for w in class_weights], dtype=torch.float,device=device)
class_weights_tensor

tensor([2.7436, 2.8330, 6.0668, 2.5528, 2.7165, 4.5248], device='cuda:0')

In [467]:
MAX_LENGTH = 512

metrics_df = pd.DataFrame(columns=["Models_name", "Precision", "Recall", "F1_score", "Accuracy"])

confusion_matrices = []

In [468]:
quantization_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_use_double_quant=True,

    bnb_4bit_quant_type="nf4",

    bnb_4bit_compute_dtype=torch.bfloat16,

)

In [469]:
models_list=models_list_3

models_name=models_name_3

x=models_list[0]

In [470]:
token=""

import huggingface_hub

if "gemma" in x or "Phi" in x:

  huggingface_hub.login(token=token)

In [471]:
USE_LORA=False

In [ ]:
!pip install -q sentence_transformers

In [472]:
from transformers import AutoModel
from sentence_transformers import SentenceTransformer

In [473]:


torch.cuda.empty_cache()



print(x)
use_gpu = torch.cuda.is_available()
embedding_model = SentenceTransformer(
        x,
        trust_remote_code=True,
        device="cpu" if not use_gpu else "cuda",
        model_kwargs={
            "torch_dtype": torch.bfloat16 if use_gpu else torch.float32,
            # "attn_implementation": "sdpa"
        },
        # vector_dim must be 12288, 1024, 512, 256
        ## 1024 is recommended
        # set is_text_encoder 'True', if you do not encode image
        # config_kwargs={"is_text_encoder": True, "vector_dim": 1024},
    )
    # We can reduce the max_seq_length from the default of 2048 for faster encoding
embedding_model.max_seq_length = MAX_LENGTH

jinaai/jina-embeddings-v3


In [474]:
def find_target_modules(model):
    # Initialize a Set to Store Unique Layers
    unique_layers = set()

    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear4bit'
        if "Linear4bit" in str(type(module)):
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]

            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)
find_target_modules(embedding_model)

[]

In [475]:
# a=

In [476]:
embedding_model.eval() # embedding_model in training mode (dropout modules are activated)

if USE_LORA:

  # enable gradient check pointing

  embedding_model.gradient_checkpointing_enable()

  target_modules_dict={

      "microsoft/Phi-3.5-mini-instruct":'all-linear',
      "unsloth/Llama-3.2-1B-Instruct":["q_proj","k_proj","v_proj"],
      "unsloth/Qwen2.5-3B-Instruct-bnb-4bit":["q_proj","k_proj","v_proj"],
  }

  target_modules= target_modules_dict.get(x,["q_proj"])



  # enable quantized training

  embedding_model = prepare_model_for_kbit_training(embedding_model)

  config = LoraConfig(

    r=8,

    lora_alpha=32,

    # target_modules=["k_proj"],

    target_modules = target_modules,#TODOOOOOOOOOOOOO

    lora_dropout=0.1,

    bias="none",

    task_type="SEQ_CLS",

  )

  # LoRA trainable version of model

  embedding_model = get_peft_model(embedding_model, config)



  # trainable parameter count

  embedding_model.print_trainable_parameters()

device="cuda" if torch.cuda.is_available() else 'cpu'

In [477]:
device

'cuda'

In [478]:
embedding_model = embedding_model.to(device)

for param in embedding_model.parameters():
    param.data = param.data.contiguous()


In [479]:

# Custom Dataset for batching
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize a single example
        return self.texts[idx]


In [480]:

# Batch extraction function
def extract_embeddings(texts, model, batch_size=32):
    dataset = TextDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    embeddings = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting embeddings"):
            # Move batch to device
            # input_ids = torch.Tensor(batch['input_ids']).squeeze().to(device)
            # attention_mask = torch.Tensor(batch['attention_mask']).squeeze().to(device)

            # Forward pass through the model
            cls_embeddings = model.encode(batch)
            embeddings.append(cls_embeddings)

    return np.vstack(embeddings)

In [481]:
# Extract embeddings for train and validation datasets
x_train_embeddings = extract_embeddings(x_train, embedding_model, batch_size=32)

Extracting embeddings: 100%|██████████| 382/382 [06:42<00:00,  1.05s/it]


In [487]:
np.save("x_train_embeddings.npy",x_train_embeddings)

In [482]:
# Flatten labels for XGBoost (multi-label classification)
y_train_flat = np.array(y_train)

In [ ]:
import xgboost as xgb

In [ ]:
# xgb_models = {}

# for i, label in enumerate(label_columns):
#     print(f"Training XGBoost for label: {label}")
#     # Use the computed class weights for the positive class
#     scale_pos_weight = class_weights_tensor[i].item()
#     # Initialize and train XGBoost classifier
#     xgb_model = xgb.XGBClassifier(
#         objective='binary:logistic',
#         eval_metric='logloss',
#         use_label_encoder=False,
#         n_estimators=100,
#         learning_rate=0.1,
#         max_depth=6,
#         tree_method='hist',
#         scale_pos_weight=scale_pos_weight,  # Apply the class weight here
#         device=device
#     )
#     xgb_model.fit(x_train_embeddings, y_train_flat[:, i])
#     xgb_models[label] = xgb_model

In [ ]:
# Save the XGBoost models if needed
# for label, model in xgb_models.items():
#     model.save_model(f"xgb_model_{label}.json")

In [ ]:
# import pickle
# # Save the XGBoost models if needed
# for label, model in xgb_models.items():
#     with open(f"xgb_model_{label}.pkl", 'wb') as f:
#       pickle.dump(model, f)

In [ ]:
# !zip xgb_models_pkl.zip xgb_model_*.pkl

In [483]:
from sklearn.svm import SVC

In [484]:
svm_models = {}

for i, label in enumerate(label_columns):
    print(f"Training SVM for label: {label}")

    # Use the computed class weights for balancing the classes
    class_weight = {0: 1, 1: class_weights_tensor[i].item()}  # Adjust the class weights

    svm_model = SVC(class_weight={0: 1, 1: class_weights_tensor[i].item()})

    svm_model.fit(x_train_embeddings, y_train_flat[:, i])
    svm_models[label] = svm_model

Training SVM for label: anger
Training SVM for label: disgust


KeyboardInterrupt: 

In [ ]:
import pickle
# Save the svm models if needed
for label, model in svm_models.items():
    with open(f"svm_model_{label}.pkl", 'wb') as f:
      pickle.dump(model, f)

In [ ]:
!zip svm_models_pkl.zip svm_model_*.pkl

In [ ]:
kind="dev"

In [ ]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)
dev_data=all_csv
dev_data=dev_data.fillna(0)
float_columns = dev_data.select_dtypes(include=['float64']).columns

# Convert those columns to int
dev_data[float_columns] = dev_data[float_columns].astype('int')
dev_data = dev_data[dev_data['clean_message'].isnull()==False]
dev_data.reset_index()


In [ ]:
x_dev, y_dev = dev_data['clean_message'].values.tolist(), dev_data[label_columns].values.tolist()

In [ ]:
y_dev_df = pd.DataFrame(y_dev, columns=label_columns)

In [ ]:
x_dev_embeddings = extract_embeddings(x_dev, embedding_model, batch_size=32)

In [ ]:
y_dev_flat = np.array(y_dev)

In [ ]:
# Predict labels for the test data
test_predictions = {}
for label in label_columns:
    print(f"Predicting for label: {label}")
    test_predictions[label] = svm_models[label].predict(x_dev_embeddings)

In [ ]:
predictions_df = pd.DataFrame(test_predictions)
dev_data.reset_index(inplace=True)

In [ ]:
output_df = pd.concat([dev_data[['id','lang']], predictions_df], axis=1)

In [ ]:
# Create directory if it doesn't exist
output_dir = task
os.makedirs(output_dir, exist_ok=True)

# Iterate through the unique languages in the 'lang' column
for lang in output_df['lang'].unique():
    # Filter rows for the current language
    lang_df = output_df[output_df['lang'] == lang]
    lang_df.drop(columns=['lang'],inplace=True)

    # Define the output file path for the current language
    lang_csv_file = os.path.join(output_dir, f"pred_{lang}.csv")

    # Sort the DataFrame by the 'id' column
    lang_df.sort_values(by='id', inplace=True)

    # Save the filtered DataFrame to a CSV file
    lang_df.to_csv(lang_csv_file, index=False)

    # Print a confirmation message
    print(f"Predictions for language '{lang}' saved to {lang_csv_file}")

In [ ]:
!zip -r {task}.zip {task}